# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.5
Scikit-Learn 1.0.2
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

##  EfficientNetB0

In [3]:
from tensorflow.keras.applications import EfficientNetB0

conv_model = EfficientNetB0(weights='imagenet', include_top=False)
conv_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, None, None, 3 7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, None, None, 3 0           normalization[0][0]              
_____________________________________________________________________________________

In [4]:
import matplotlib.pyplot as plt
def plot_history(history, yrange):
    '''Plot loss and accuracy as a function of the epoch,
    for the training and validation datasets.
    '''
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    # Plot training and validation accuracy per epoch
    plt.plot(epochs, acc)
    plt.plot(epochs, val_acc)
    plt.title('Training and validation accuracy')
    plt.ylim(yrange)
    
    # Plot training and validation loss per epoch
    plt.figure()

    plt.plot(epochs, loss)
    plt.plot(epochs, val_loss)
    plt.title('Training and validation loss')
    
    plt.show()

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

def generators(shape): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    path_dir = '../dataset_mel_final/train'
    parth_dir_test = '../dataset_mel_final/test'

    datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

    train_dataset = datagen.flow_from_directory(
        path_dir,
        target_size=(150,150),
        shuffle=True,
        subset='training',
        batch_size = 64,
        class_mode = 'binary'
    )
    
    val_dataset = datagen.flow_from_directory(
        path_dir,
        target_size=(150,150),
        subset='validation',
        batch_size = 64,
        class_mode = 'binary'
    )

    test_dataset = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        class_mode = 'binary')
    return train_dataset, val_dataset, test_dataset

In [6]:
train_dataset, val_dataset, test_dataset = generators((150,150))

Found 1102 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [7]:
conv_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150,150,3))
conv_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 150, 150, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 150, 150, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization_1[0][0]            
_____________________________________________________________________________________

In [8]:
for layer in conv_model.layers[:-2]:
    layer.trainable = False

In [9]:
# flatten the output of the convolutional part: 
from tensorflow import keras

classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5
classifier.add(conv_model) 
#Adding a convolutional layer
classifier.add(Conv2D(16, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 16, activation = 'relu'))
#classifier.add(Dropout(0.01))
classifier.add(Dense(units = 1, activation= 'sigmoid'))


In [10]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 5, 5, 1280)        4049571   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 16)          184336    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 2, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 4,234,964
Trainable params: 187,953
Non-trainable params: 4,047,011
________________________________________

In [11]:
classifier.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-5),
                  metrics=['acc'])

C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=10)
mc = ModelCheckpoint('best_efficientnetb0.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [13]:
history = classifier.fit_generator(
    train_dataset, 
    validation_data = val_dataset,
    callbacks = [tensorboard_callback,es,mc,reduceLR],
    epochs=35
)

C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1969: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/35
18/18 [==============================] - 16s 471ms/step - loss: 0.9168 - acc: 0.5000 - val_loss: 0.6995 - val_acc: 0.5000
Epoch 2/35
18/18 [==============================] - 4s 213ms/step - loss: 0.7222 - acc: 0.5000 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 3/35
18/18 [==============================] - 4s 213ms/step - loss: 0.7003 - acc: 0.4991 - val_loss: 0.6935 - val_acc: 0.5000
Epoch 4/35
18/18 [==============================] - 4s 211ms/step - loss: 0.6947 - acc: 0.4946 - val_loss: 0.6941 - val_acc: 0.5000
Epoch 5/35
18/18 [==============================] - 4s 212ms/step - loss: 0.6948 - acc: 0.4918 - val_loss: 0.6945 - val_acc: 0.5000
Epoch 6/35
18/18 [==============================] - 4s 212ms/step - loss: 0.6938 - acc: 0.4973 - val_loss: 0.6941 - val_acc: 0.5000
Epoch 7/35
18/18 [==============================] - 4s 213ms/step - loss: 0.6929 - acc: 0.5100 - val_loss: 0.6938 - val_acc: 0.5000
Epoch 8/35
18/18 [==============================] - 4s 216ms/step - loss: 0

In [ ]:
plot_history(history, yrange=(0,1))

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
classifier.load_weights('best_efficientnetb0.h5')

In [ ]:
classifier.summary()

In [ ]:
score = classifier.evaluate(test_dataset, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

In [ ]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d